In [1]:
import tensorflow as tf
from PIL import Image
import multiprocessing as mp
import os

2023-06-06 17:31:32.794786: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-06-06 17:31:32.848700: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-06 17:31:33.513193: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
from keras import backend as K
# tf.config.set_visible_devices([], 'GPU') # für CPU usage
config = tf.compat.v1.ConfigProto(device_count = {'GPU': len(tf.config.list_physical_devices('GPU')), # sollte 1 sein, weil wir nur eine GPU haben
                                                  'CPU': mp.cpu_count()})                             # sollte 64 sein, weil wir 64 CPU Kerne haben
sess = tf.compat.v1.Session(config=config) 
K.set_session(sess)

2023-06-06 17:31:35.407918: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1635] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 78918 MB memory:  -> device: 0, name: NVIDIA A100 80GB PCIe, pci bus id: 0000:c0:00.0, compute capability: 8.0


### Paths

In [3]:
DATASET_PATH = "/scratch/fetaiajshe/dataset/"

TEST_PATH = DATASET_PATH + "test/"
TRAIN_PATH = DATASET_PATH + "train/"

TEST_PATH_N = TEST_PATH + "n/"
TEST_PATH_Y = TEST_PATH + "y/"
TRAIN_PATH_N = TRAIN_PATH + "n/"
TRAIN_PATH_Y = TRAIN_PATH + "y/"

### Data Generation

In [4]:
TARGET_SIZE = (1250, 1250)
EPOCHS = 10
STEPS_PER_EPOCH = 100
VAL_STEPS = 50
BATCH_SIZE = 20

In [5]:
# NO DATA AUGMENTATION
datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)
train_gen = datagen.flow_from_directory(TRAIN_PATH,
                                       target_size = TARGET_SIZE,
                                       batch_size = BATCH_SIZE,
                                       class_mode = 'binary')

Found 2713 images belonging to 2 classes.


### Base Model

In [6]:
def base_model():
    model = tf.keras.models.Sequential()
    # CNN Block 1
    model.add(tf.keras.layers.Conv2D(32, (3,3), activation = 'relu', input_shape=(TARGET_SIZE[0],TARGET_SIZE[1],3)))
    model.add(tf.keras.layers.MaxPooling2D((2, 2)))
    # CNN Block 2
    model.add(tf.keras.layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(tf.keras.layers.MaxPooling2D((2, 2)))
    # CNN Block 3
    model.add(tf.keras.layers.Conv2D(128, (3, 3), activation='relu'))
    model.add(tf.keras.layers.MaxPooling2D((2, 2)))
    # Classification Block
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(256, activation= 'relu'))
    model.add(tf.keras.layers.Dense(1, activation='sigmoid'))
    return model
baseline_model = base_model()
baseline_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
baseline_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 1248, 1248, 32)    896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 624, 624, 32)     0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 622, 622, 64)      18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 311, 311, 64)     0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 309, 309, 128)     73856     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 154, 154, 128)    0

2023-06-06 17:31:35.739807: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1635] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 78918 MB memory:  -> device: 0, name: NVIDIA A100 80GB PCIe, pci bus id: 0000:c0:00.0, compute capability: 8.0


In [7]:
baseline_model.fit(train_gen, epochs=EPOCHS)

Epoch 1/10


2023-06-06 17:31:39.875684: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]
2023-06-06 17:31:42.589938: W tensorflow/core/kernels/gpu_utils.cc:50] Failed to allocate memory for convolution redzone checking; skipping this check. This is benign and only means that we won't check cudnn for out-of-bounds reads and writes. This message will only be printed once.
2023-06-06 17:31:43.986369: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:424] Loaded cuDNN version 8600
2023-06-06 17:31:44.753881: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-06-06 17:31:46.922529: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:637] TensorFloat-32 will be used for the ma

32/85 [==========>...................] - ETA: 1:20 - loss: 12.8988 - accuracy: 0.9365